In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.19 pyspark-shell'

from time import sleep

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [4]:
spark = SparkSession. \
    builder.config("spark.driver.host", "localhost"). \
    appName("Spark DSL"). \
    master("local"). \
    config("spark.sql.legacy.timeParserPolicy", "LEGACY"). \
    getOrCreate()

In [5]:
movies_df = spark.read.json("data/movies")

# Aggregations

## Counting

In [6]:
movies_df.printSchema()

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)



In [6]:
# NULL значения не участвуют в подсчете
all_movies_count_df = movies_df.selectExpr("count(Major_Genre)")


In [7]:
all_movies_count_df.show()

+------------------+
|count(Major_Genre)|
+------------------+
|              2926|
+------------------+



In [8]:
all_movies_count_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(Major_Genre#14)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=46]
      +- HashAggregate(keys=[], functions=[partial_count(Major_Genre#14)])
         +- FileScan json [Major_Genre#14] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [9]:
# NULL значения не участвуют в подсчете
genres_count_df = movies_df.select(count(col("Major_Genre")))
genres_count_df.show()

+------------------+
|count(Major_Genre)|
+------------------+
|              2926|
+------------------+



In [10]:
genres_count_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(Major_Genre#14)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=91]
      +- HashAggregate(keys=[], functions=[partial_count(Major_Genre#14)])
         +- FileScan json [Major_Genre#14] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [11]:
# NULL значения включены в подсчет строк
genres_count_df = movies_df.select(count(col("*")).alias("Общее колличество строк"))
genres_count_df.show()

+-----------------------+
|Общее колличество строк|
+-----------------------+
|                   3201|
+-----------------------+



In [12]:
genres_count_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(1)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=145]
      +- HashAggregate(keys=[], functions=[partial_count(1)])
         +- FileScan json [] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<>




In [11]:
# NULL значения включены в подсчет строк
genres_count_df_v2 = movies_df.selectExpr('count(*) as Total_Rows')
genres_count_df_v2.show()

+----------+
|Total_Rows|
+----------+
|      3201|
+----------+



In [12]:
# NULL значения включены в подсчет строк, `.count()` -- Action
genres_count_number = movies_df.select("Major_Genre").count()
print("Общее колличество строк:", genres_count_number)

In [11]:
# Подсчет уникальных значений
unique_genres_df = movies_df.select(F.countDistinct(F.col("Major_Genre")).alias("Уникальное число жанров"))
unique_genres_df.show()

+-----------------------+
|Уникальное число жанров|
+-----------------------+
|                     12|
+-----------------------+



In [14]:
unique_genres_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(distinct Major_Genre#14)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=245]
      +- HashAggregate(keys=[], functions=[partial_count(distinct Major_Genre#14)])
         +- HashAggregate(keys=[Major_Genre#14], functions=[])
            +- Exchange hashpartitioning(Major_Genre#14, 200), ENSURE_REQUIREMENTS, [plan_id=241]
               +- HashAggregate(keys=[Major_Genre#14], functions=[])
                  +- FileScan json [Major_Genre#14] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [14]:
unique_genres_df_v2 = movies_df.selectExpr("count(DISTINCT Major_Genre) as Number_Of_Unique_Genres")
print("COUNT DISTINCT Genre with Expression")
unique_genres_df_v2.show()

# Math aggregations

In [15]:
# min/max
max_rating_df = movies_df \
  .select(
    max(col("IMDB_Rating")).alias("max value of IMDB Rating"),
    min(col("IMDB_Rating")).alias("min value of IMDB Rating")
  )
max_rating_df.show()

In [16]:
max_rating_df_v2 = movies_df \
  .selectExpr(
    "max(IMDB_Rating) max_rating",
    "min(IMDB_Rating) as min_rating"
  )
max_rating_df_v2.show()

In [17]:
# сумма значений в столбце
us_industry_total_df = movies_df \
  .select(
    sum(col("US_Gross")) \
      .alias("Sum US Gross")
  )
us_industry_total_df.show()

In [18]:
us_industry_total_df_v2 = movies_df.selectExpr("sum(US_Gross) as Sum_US_Gross")
us_industry_total_df_v2.show()

In [19]:
# avg
avg_rt_rating_df = movies_df.select(avg(col("Rotten_Tomatoes_Rating")).alias("Average Rotten Tomatoes Rating"))
avg_rt_rating_df.show()

In [19]:
# mean/standard dev
rt_stats_df = movies_df.agg(
    mean(col("Rotten_Tomatoes_Rating")) \
      .alias("Mean of Rotten Tomatoes Rating"),
    stddev(col("Rotten_Tomatoes_Rating")) \
      .alias("Standard Deviation in Rotten Tomatoes Rating")
)
rt_stats_df.show()

+------------------------------+--------------------------------------------+
|Mean of Rotten Tomatoes Rating|Standard Deviation in Rotten Tomatoes Rating|
+------------------------------+--------------------------------------------+
|             54.33692373976734|                           28.07659263787602|
+------------------------------+--------------------------------------------+



# Grouping

In [22]:
# Группировка по колонке. NULL значения так же образуют группу
count_by_genre_df = movies_df. \
    groupBy("Major_Genre"). \
    count()

count_by_genre_df.show()

+-------------------+-----+
|        Major_Genre|count|
+-------------------+-----+
|          Adventure|  274|
|               NULL|  275|
|              Drama|  789|
|        Documentary|   43|
|       Black Comedy|   36|
|  Thriller/Suspense|  239|
|            Musical|   53|
|    Romantic Comedy|  137|
|Concert/Performance|    5|
|             Horror|  219|
|            Western|   36|
|             Comedy|  675|
|             Action|  420|
+-------------------+-----+



In [23]:
count_by_genre_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Major_Genre#14], functions=[count(1)])
   +- Exchange hashpartitioning(Major_Genre#14, 200), ENSURE_REQUIREMENTS, [plan_id=436]
      +- HashAggregate(keys=[Major_Genre#14], functions=[partial_count(1)])
         +- FileScan json [Major_Genre#14] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Major_Genre:string>




In [23]:
# Операции над группами
avg_rating_by_genre_df = movies_df. \
    groupBy(col("Major_Genre")). \
    avg("IMDB_Rating")

avg_rating_by_genre_df.show()

In [24]:
avg_rating_by_genre_df.explain()

In [16]:
# Множетственные операции над группами
aggregations_by_genre_df = movies_df. \
  groupBy(col("Major_Genre")). \
  agg(
    # use strings here for column names
    count("*") \
      .alias("N Movies"),
    avg("IMDB_Rating") \
      .alias("Average Rating")
  )

aggregations_by_genre_df.show()

+-------------------+--------+------------------+
|        Major_Genre|N Movies|    Average Rating|
+-------------------+--------+------------------+
|          Adventure|     274| 6.345019920318729|
|               NULL|     275|  6.50082644628099|
|              Drama|     789| 6.773441734417339|
|        Documentary|      43| 6.997297297297298|
|       Black Comedy|      36|6.8187500000000005|
|  Thriller/Suspense|     239| 6.360944206008582|
|            Musical|      53|             6.448|
|    Romantic Comedy|     137| 5.873076923076922|
|Concert/Performance|       5|             6.325|
|             Horror|     219|5.6760765550239185|
|            Western|      36| 6.842857142857142|
|             Comedy|     675| 5.853858267716529|
|             Action|     420| 6.114795918367349|
+-------------------+--------+------------------+



In [17]:
aggregations_by_genre_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Major_Genre#14], functions=[count(1), avg(IMDB_Rating#11)])
   +- Exchange hashpartitioning(Major_Genre#14, 200), ENSURE_REQUIREMENTS, [plan_id=372]
      +- HashAggregate(keys=[Major_Genre#14], functions=[partial_count(1), partial_avg(IMDB_Rating#11)])
         +- FileScan json [IMDB_Rating#11,Major_Genre#14] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/movies], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<IMDB_Rating:double,Major_Genre:string>




# Sorting

In [12]:
# Числа, строки и даты можно сортировать
best_movies_df = movies_df \
  .selectExpr("IMDB_Rating", "Title") \
  .orderBy(col("IMDB_Rating").desc())
best_movies_df.show(10, False)

+-----------+-------------------------------+
|IMDB_Rating|Title                          |
+-----------+-------------------------------+
|9.2        |The Godfather                  |
|9.2        |The Shawshank Redemption       |
|9.1        |Inception                      |
|9.0        |The Godfather: Part II         |
|8.9        |Pulp Fiction                   |
|8.9        |12 Angry Men                   |
|8.9        |One Flew Over the Cuckoo's Nest|
|8.9        |Schindler's List               |
|8.9        |The Dark Knight                |
|8.9        |Toy Story 3                    |
+-----------+-------------------------------+
only showing top 10 rows



In [28]:
# Сортировка с NULLS_FIRST
proper_worst_movies_df = movies_df \
  .selectExpr("IMDB_Rating", "Title") \
  .orderBy(col("IMDB_Rating").desc_nulls_first())

proper_worst_movies_df.show(10, False)

# Упражнения

1. Получить общую кассу всех фильмов (сколько фильмы заработали все вместе)
1. Посчитать общее колличество режиссеров (один режиссер мог снять несколько фильмов)
1. Вычислить среднее (mean) и стандартное отклонение (stddev) для прибыли с американского проката (US_Gross)
1. Для каждого режиссера вычислить средний (avg) рейтинг на IMDB и среднюю (avg) прибыль в американском прокате 
1. Вычислить среднюю (avg) разность между IMDB и Rotten Tomatoes рейтингами

# Joins

In [3]:
guitars_df = spark.read.json("data/guitars")
guitar_players_df = spark.read.json("data/guitarPlayers")
bands_df = spark.read.json("data/bands")
guitars_df.printSchema()
guitar_players_df.printSchema()
bands_df.printSchema()

root
 |-- guitarType: string (nullable = true)
 |-- id: long (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)

root
 |-- band: long (nullable = true)
 |-- guitars: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)

root
 |-- hometown: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- year: long (nullable = true)



In [4]:
 guitar_players_df.show()

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   2| [1, 5]|  2|Eric Clapton|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



In [5]:
bands_df.show()

+-----------+---+------------+----+
|   hometown| id|        name|year|
+-----------+---+------------+----+
|     Sydney|  1|       AC/DC|1973|
|     London|  0|Led Zeppelin|1968|
|Los Angeles|  3|   Metallica|1981|
|  Liverpool|  4| The Beatles|1960|
+-----------+---+------------+----+



In [6]:
join_condition = guitar_players_df.band == bands_df.id
join_condition

Column<'(band = id)'>

In [7]:
# INNER JOIN: все строки, которые есть и в "правом", и в "левом" множестве, для которых верно условие

guitarists_bands_df = guitar_players_df \
  .join(
    bands_df,
    join_condition,
    # "inner" # <-- 'inner' == Default
  )

guitarists_bands_df.show()

+----+-------+---+------------+-----------+---+------------+----+
|band|guitars| id|        name|   hometown| id|        name|year|
+----+-------+---+------------+-----------+---+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
+----+-------+---+------------+-----------+---+------------+----+



In [8]:
guitarists_bands_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [band#24L], [id#41L], Inner, BuildLeft, false
   :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=129]
   :  +- Filter isnotnull(band#24L)
   :     +- FileScan json [band#24L,guitars#25,id#26L,name#27] Batched: false, DataFilters: [isnotnull(band#24L)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/guitarPlayers], PartitionFilters: [], PushedFilters: [IsNotNull(band)], ReadSchema: struct<band:bigint,guitars:array<bigint>,id:bigint,name:string>
   +- Filter isnotnull(id#41L)
      +- FileScan json [hometown#40,id#41L,name#42,year#43L] Batched: false, DataFilters: [isnotnull(id#41L)], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/bands], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<hometown:string,id:bigint,name:string,year:bigint>




In [20]:
join_condition

Column<'(band = id)'>

In [55]:
# Привести колонку "name" к верхнему регистру
# Обратите внимание, что `guitarists_bands_df` строится на базе `bands_df`, поэтому можно использовать его колонки
guitarists_bands_upper_df = guitarists_bands_df.select(upper(bands_df.name))
guitarists_bands_upper_df.show()

In [56]:
guitarists_bands_upper_df.explain()

In [35]:
# left outer = все строки с лева, для которых условие верно. Если строки с таким ключом нет в правом множестве, то поставить null
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "left_outer"
  ) \
  .show()

+----+-------+---+------------+-----------+----+------------+----+
|band|guitars| id|        name|   hometown|  id|        name|year|
+----+-------+---+------------+-----------+----+------------+----+
|   0|    [0]|  0|  Jimmy Page|     London|   0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     Sydney|   1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|       NULL|NULL|        NULL|NULL|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|   3|   Metallica|1981|
+----+-------+---+------------+-----------+----+------------+----+



In [36]:
# Right outer = все строки с права, для которых условие верно. Если строки с таким ключом нет в левом множестве, то поставить null
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "right_outer"
  ) \
  .show()

+----+-------+----+------------+-----------+---+------------+----+
|band|guitars|  id|        name|   hometown| id|        name|year|
+----+-------+----+------------+-----------+---+------------+----+
|   0|    [0]|   0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   1|    [1]|   1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   3|    [3]|   3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
|NULL|   NULL|NULL|        NULL|  Liverpool|  4| The Beatles|1960|
+----+-------+----+------------+-----------+---+------------+----+



In [37]:
# full outer join = все строки справа и с лева, для которых верно условие, поставить NULL если строки с соответствующим ключом нет в правом или в левом множестве
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "outer"
  ) \
  .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [band#636L], [id#653L], FullOuter
   :- Sort [band#636L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(band#636L, 200), ENSURE_REQUIREMENTS, [plan_id=965]
   :     +- FileScan json [band#636L,guitars#637,id#638L,name#639] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/guitarPlayers], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<band:bigint,guitars:array<bigint>,id:bigint,name:string>
   +- Sort [id#653L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#653L, 200), ENSURE_REQUIREMENTS, [plan_id=966]
         +- FileScan json [hometown#652,id#653L,name#654,year#655L] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/notebooks/data/bands], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<hometown:string,id:bigint,name:string,year:bigint>




In [61]:
# Соединить по одной колонке
guitar_players_df \
  .join(
    bands_df,
    "id"
  ) \
  .show()

In [26]:
# left semi joins = все элементы в левом множестве, для которых есть соответствующий элемент в правом множестве для которого верно условие
# Похоже на фильтр
# Эквивалентный SQL (колонок из правого множества нет): select * from guitar_players WHERE EXISTS (...)
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "left_semi"
  ) \
  .show()

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



In [27]:
# Исключить из левого множества каждую строку, для ключа которой есть строка с таким ключом в правом множестве
# Похоже на разность множеств
guitar_players_df \
  .join(
    bands_df,
    join_condition,
    "left_anti"
  ) \
  .show()

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+



In [63]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://postgres:5432/spark"
user = "docker"
password = "docker"

In [64]:
def read_table(table_name):
    return spark.read. \
        format("jdbc"). \
        option("driver", driver). \
        option("url", url). \
        option("user", user). \
        option("password", password). \
        option("dbtable", "public." + table_name). \
        load()

employees_df = read_table("employees")
salaries_df = read_table("salaries")
dept_managers_df = read_table("dept_manager")
dept_emp_df = read_table("dept_emp")
departments_df = read_table("departments")

In [65]:
all_df = employees_df \
  .join(dept_emp_df, "emp_no") \
  .join(F.broadcast(departments_df), "dept_no") \
  .join(salaries_df, "emp_no")

all_df.show(10)
all_df.explain()

# Задания

Загрузить следующие таблицы из базы данных Postgres: `employees`, `salaries`, `dept_emp`

1. Получить список всех сотрудников и их максимальные зарплаты
1. Получить список всех сотрудников, кто никогда не был менеджером
1. Для каждого сотрудника, найти разницу между их зарплатой (текущей/последней) и максимальной зарплатой в их отделе